### EDA Playground

- GUI 환경에서 dataset classes 선택시 해당하는 class별로 볼 수 있음
- anormaly - normal pair data를 찾아서 묶어준다음 idx별로 볼 수 있게 만듬.
- 스펙트로그램을 볼떄 n_fft 사이즈를 조절가능함. html로 따지면 range와 number input으로 조작가능
- 소리를 들을 수 있도록 재생 버튼도 있어야함


In [1]:
import os
import random
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display, HTML
import warnings
import ipywidgets as widgets
from ipywidgets import interact, fixed
from skimage.metrics import structural_similarity as ssim
import base64
from io import BytesIO
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
import uuid
from scipy.spatial.distance import cosine



# Suppress warnings from librosa (optional)
warnings.filterwarnings('ignore')

# -----------------------------
# Configuration Parameters
# -----------------------------
DATASETS_DIR = "../../../datasets/dev"  # Path to the datasets directory
CLASS_NAMES = [name for name in os.listdir(DATASETS_DIR) if os.path.isdir(os.path.join(DATASETS_DIR, name))]
N_FFT = 160

# -----------------------------
# Step 1: Load Dataset
# -----------------------------
def load_dataset(attributes_file, datasets_dir, class_name):
    if not os.path.isfile(attributes_file):
        raise FileNotFoundError(f"Attributes file not found: {attributes_file}")

    df = pd.read_csv(attributes_file)
    filenames = df['file_name'].tolist()
    labels = ['anomaly' if 'anomaly' in name.lower() else 'normal' for name in filenames]
    
    # 파일 경로를 생성할 때 datasets_dir, class_name, 파일명을 합침
    file_paths = [os.path.join(datasets_dir, f) for f in filenames]

    return file_paths, labels

# -----------------------------
# Step 2: Compute Spectrogram
# -----------------------------
def compute_spectrogram(y, n_fft, hop_length):
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)
    return S_mag

# -----------------------------
# Step 3: Find Corresponding Normal File for Each Anomaly Based on Exact Code Name and File Number
# -----------------------------
def find_matching_normal_file(anomaly_path, normal_paths):
    anomaly_filename = os.path.basename(anomaly_path)
    anomaly_parts = anomaly_filename.split("_")

    # Adjust indices according to your filename structure
    anomaly_number = anomaly_parts[5] if len(anomaly_parts) > 5 else None
    anomaly_code = anomaly_parts[6] if len(anomaly_parts) > 6 else None  # Adjusted index

    if anomaly_number is None:
        print(f"Unexpected file name format: {anomaly_filename}. No direct match found for anomaly file: {anomaly_path}")
        return None

    for normal_path in normal_paths:
        normal_filename = os.path.basename(normal_path)
        normal_parts = normal_filename.split("_")

        normal_number = normal_parts[5] if len(normal_parts) > 5 else None
        normal_code = normal_parts[6] if len(normal_parts) > 6 else None  # Adjusted index

        if anomaly_code is None:
            if anomaly_number == normal_number:
                return normal_path
        else:
            if anomaly_number == normal_number and anomaly_code == normal_code:
                return normal_path

    print(f"No direct match found for anomaly file: {anomaly_path}")
    return None



# -----------------------------
# Step 4: Find Most Similar Normal for Anomaly Using cosine
# -----------------------------
def compute_low_res_spectrogram(y, sr, n_fft=160, hop_length=80, target_shape=(32, 32)):
    # 스펙트로그램 계산
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    # 저해상도 스펙트로그램으로 다운샘플링
    S_resized = librosa.util.fix_length(S, size=target_shape[0], axis=0)
    S_resized = librosa.util.fix_length(S_resized, size=target_shape[1], axis=1)
    return S_resized

def find_most_similar_normal_cosine(anomaly_spectrogram, normal_spectrograms):
    max_similarity = -1
    most_similar_normal = None

    for normal_spectrogram, path in normal_spectrograms:
        # 코사인 유사도 계산
        similarity = 1 - cosine(anomaly_spectrogram.flatten(), normal_spectrogram.flatten())
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_normal = path

    return most_similar_normal, max_similarity

# -----------------------------
# Step 5: Plot and Play Normal and Matching Anomaly Spectrograms
# -----------------------------
def plot_and_play_normal_anomaly_pair(anom_path, norm_path, n_fft, hop_length):
    y_anomaly, sr_anomaly = librosa.load(anom_path, sr=None)
    y_normal, sr_normal = librosa.load(norm_path, sr=None)

    duration_anomaly = len(y_anomaly) / sr_anomaly
    duration_normal = len(y_normal) / sr_normal

    S_anomaly = compute_spectrogram(y_anomaly, n_fft, hop_length)
    S_normal = compute_spectrogram(y_normal, n_fft, hop_length)

    # Convert spectrograms to decibel scale
    S_db_anomaly = librosa.amplitude_to_db(S_anomaly, ref=np.max)
    S_db_normal = librosa.amplitude_to_db(S_normal, ref=np.max)

    # Generate data URIs for spectrogram images
    def spectrogram_to_data_uri(S_db, hop_length, sr, duration, title):
        fig, ax = plt.subplots(figsize=(10, 3))

        # Plot spectrogram
        img = librosa.display.specshow(
            S_db,
            sr=sr,
            hop_length=hop_length,
            x_axis='time',
            y_axis='linear',
            ax=ax
        )

        # Set x-axis limits to match audio duration
        ax.set_xlim(0, duration)

        # Remove margins and ticks
        ax.set_title(title)
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_yticklabels([])
        ax.set_xticklabels([])

        # Remove whitespace around the plot
        plt.tight_layout(pad=0)
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

        # Save the figure to a BytesIO object
        buf = BytesIO()
        fig.savefig(buf, format='png', dpi=100, bbox_inches='tight', pad_inches=0)
        buf.seek(0)
        # Close the figure
        plt.close(fig)
        # Encode to base64 string
        img_base64 = base64.b64encode(buf.read()).decode('utf-8')
        return 'data:image/png;base64,' + img_base64

    anomaly_img_uri = spectrogram_to_data_uri(S_db_anomaly, hop_length, sr_anomaly, duration_anomaly, 'Anomaly Spectrogram')
    normal_img_uri = spectrogram_to_data_uri(S_db_normal, hop_length, sr_normal, duration_normal, 'Normal Spectrogram')

    # Generate data URIs for audio
    def audio_to_data_uri(y, sr):
        buf = BytesIO()
        sf.write(buf, y, sr, format='WAV')
        buf.seek(0)
        # Encode to base64
        audio_base64 = base64.b64encode(buf.read()).decode('utf-8')
        return 'data:audio/wav;base64,' + audio_base64

    anomaly_audio_uri = audio_to_data_uri(y_anomaly, sr_anomaly)
    normal_audio_uri = audio_to_data_uri(y_normal, sr_normal)

    # Generate unique IDs
    anomaly_id = 'anomaly_' + str(uuid.uuid4()).replace('-', '')
    normal_id = 'normal_' + str(uuid.uuid4()).replace('-', '')

    # Generate HTML
    html_template = """
    <div style="display: flex; flex-direction: column; align-items: center;">
        <label for="global_speed_select">재생 속도 (Playback Speed):</label>
        <select id="global_speed_select" style="margin-bottom: 10px;">
            <option value="0.5">0.5x</option>
            <option value="0.75">0.75x</option>
            <option value="1.0" selected>1.0x</option>
            <option value="1.25">1.25x</option>
            <option value="1.5">1.5x</option>
            <option value="2.0">2.0x</option>
        </select>

        <div style="display: flex; align-items: center; margin-bottom: 20px;">
            <div style="position: relative; display: inline-block;">
                <img src="{anomaly_img_uri}" id="img_{anomaly_id}" style="display: block; margin: 0; padding: 0;" />
                <canvas id="cursor_canvas_{anomaly_id}" style="position: absolute; top: 0; left: 0;"></canvas>
            </div>
            <audio id="audio_{anomaly_id}" controls style="margin-left: 10px;">
                <source src="{anomaly_audio_uri}" type="audio/wav">
                Your browser does not support the audio element.
            </audio>
        </div>

        <div style="display: flex; align-items: center;">
            <div style="position: relative; display: inline-block;">
                <img src="{normal_img_uri}" id="img_{normal_id}" style="display: block; margin: 0; padding: 0;" />
                <canvas id="cursor_canvas_{normal_id}" style="position: absolute; top: 0; left: 0;"></canvas>
            </div>
            <audio id="audio_{normal_id}" controls style="margin-left: 10px;">
                <source src="{normal_audio_uri}" type="audio/wav">
                Your browser does not support the audio element.
            </audio>
        </div>
    </div>
    <script>
    (function() {{
        var audioAnomaly = document.getElementById('audio_{anomaly_id}');
        var audioNormal = document.getElementById('audio_{normal_id}');
        var canvasAnomaly = document.getElementById('cursor_canvas_{anomaly_id}');
        var canvasNormal = document.getElementById('cursor_canvas_{normal_id}');
        var imgAnomaly = document.getElementById('img_{anomaly_id}');
        var imgNormal = document.getElementById('img_{normal_id}');
        var durationAnomaly = {duration_anomaly};
        var durationNormal = {duration_normal};

        var speedSelect = document.getElementById('global_speed_select');
        speedSelect.addEventListener('change', function() {{
            var playbackRate = parseFloat(this.value);
            audioAnomaly.playbackRate = playbackRate;
            audioNormal.playbackRate = playbackRate;
        }});

        function resizeCanvas(canvas, img) {{
            canvas.width = img.naturalWidth;
            canvas.height = img.naturalHeight;
            canvas.style.width = img.width + 'px';
            canvas.style.height = img.height + 'px';
        }}

        imgAnomaly.onload = function() {{
            resizeCanvas(canvasAnomaly, imgAnomaly);
        }};
        imgNormal.onload = function() {{
            resizeCanvas(canvasNormal, imgNormal);
        }};
        window.addEventListener('resize', function() {{
            resizeCanvas(canvasAnomaly, imgAnomaly);
            resizeCanvas(canvasNormal, imgNormal);
        }});

        function drawCursor(audio, canvas, duration) {{
            var ctx = canvas.getContext('2d');
            var currentTime = audio.currentTime;
            var x = (currentTime / duration) * canvas.width;
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.beginPath();
            ctx.moveTo(x, 0);
            ctx.lineTo(x, canvas.height);
            ctx.strokeStyle = 'red';
            ctx.lineWidth = 2;
            ctx.stroke();

            if (!audio.paused) {{
                requestAnimationFrame(function() {{
                    drawCursor(audio, canvas, duration);
                }});
            }}
        }}

        audioAnomaly.addEventListener('play', function() {{
            requestAnimationFrame(function() {{
                drawCursor(audioAnomaly, canvasAnomaly, durationAnomaly);
            }});
        }});
        audioNormal.addEventListener('play', function() {{
            requestAnimationFrame(function() {{
                drawCursor(audioNormal, canvasNormal, durationNormal);
            }});
        }});

        audioAnomaly.addEventListener('pause', function() {{
            canvasAnomaly.getContext('2d').clearRect(0, 0, canvasAnomaly.width, canvasAnomaly.height);
        }});
        audioNormal.addEventListener('pause', function() {{
            canvasNormal.getContext('2d').clearRect(0, 0, canvasNormal.width, canvasNormal.height);
        }});

        audioAnomaly.addEventListener('seeked', function() {{
            drawCursor(audioAnomaly, canvasAnomaly, durationAnomaly);
        }});
        audioNormal.addEventListener('seeked', function() {{
            drawCursor(audioNormal, canvasNormal, durationNormal);
        }});
    }})();
    </script>
    """

    combined_html = html_template.format(
        anomaly_img_uri=anomaly_img_uri,
        normal_img_uri=normal_img_uri,
        anomaly_audio_uri=anomaly_audio_uri,
        normal_audio_uri=normal_audio_uri,
        anomaly_id=anomaly_id,
        normal_id=normal_id,
        duration_anomaly=duration_anomaly,
        duration_normal=duration_normal
    )

    display(HTML(combined_html))


# -----------------------------
# Interactive Widgets and Main Execution Flow
# -----------------------------
def interactive_visualization():
    class_dropdown = widgets.Dropdown(
        options=CLASS_NAMES,
        value=CLASS_NAMES[0],
        description='Class:',
        disabled=False,
    )

    n_fft_slider = widgets.IntSlider(
        value=N_FFT,
        min=32,
        max=512,
        step=2,
        description='n_fft:',
        continuous_update=False
    )

    hop_length_ratio_dropdown = widgets.Dropdown(
        options=[1/2, 1/3, 1/4],
        value=1/2,
        description='Hop Ratio:',
        disabled=False
    )

    index_input = widgets.IntText(
        value=0,
        description='Index:',
        continuous_update=True,
        disabled=False
    )

    def update_index_slider(*args):
        class_name = class_dropdown.value
        attributes_file = os.path.join(DATASETS_DIR, class_name, "attributes_00.csv")
        datasets_dir = DATASETS_DIR
        try:
            file_paths, labels = load_dataset(attributes_file, datasets_dir, class_name)
            anomaly_paths = [path for path, label in zip(file_paths, labels) if label == 'anomaly']
            index_input.max = len(anomaly_paths) - 1 if len(anomaly_paths) > 0 else 0
        except FileNotFoundError:
            index_input.max = 0

    class_dropdown.observe(update_index_slider, names='value')
    update_index_slider()

    def visualize(class_name, n_fft, hop_ratio, pair_index):
        hop_length = int(n_fft * hop_ratio)
        attributes_file = os.path.join(DATASETS_DIR, class_name, "attributes_00.csv")
        datasets_dir = DATASETS_DIR

        try:
            file_paths, labels = load_dataset(attributes_file, datasets_dir, class_name)
        except FileNotFoundError as e:
            print(e)
            return

        normal_paths = [path for path, label in zip(file_paths, labels) if label == 'normal']
        anomaly_paths = [path for path, label in zip(file_paths, labels) if label == 'anomaly']

        if pair_index >= len(anomaly_paths):
            print("Index out of range.")
            return

        anomaly_path = anomaly_paths[pair_index]
        print(f"Current anomaly file: {anomaly_path}")
        matching_normal_path = find_matching_normal_file(anomaly_path, normal_paths)

        if not matching_normal_path:
            print(f"No direct match found for anomaly file: {anomaly_path}")
            normal_spectrograms = [(compute_spectrogram(librosa.load(path, sr=None)[0], n_fft, hop_length), path) for path in normal_paths]
            y_anomaly, _ = librosa.load(anomaly_path, sr=None)
            S_anomaly = compute_spectrogram(y_anomaly, n_fft, hop_length)
            matching_normal_path, max_ssim = find_most_similar_normal_cosine(S_anomaly, normal_spectrograms)
            print(f"Most similar normal file: {matching_normal_path} (similarity: {max_ssim})")
        else:
            print(f"Matching normal file: {matching_normal_path}")

        plot_and_play_normal_anomaly_pair(anomaly_path, matching_normal_path, n_fft, hop_length)

    interact(
        visualize,
        class_name=class_dropdown,
        n_fft=n_fft_slider,
        hop_ratio=hop_length_ratio_dropdown,
        pair_index=index_input
    )

if __name__ == "__main__":
    interactive_visualization()

interactive(children=(Dropdown(description='Class:', options=('ToyCar', 'gearbox', 'valve', 'bearing', 'slider…